In [1]:
import gdax
import pandas as pd
import time
import os.path
import requests
from shutil import copyfile
import cProfile
import datetime
import talib
import numpy as np

n = 4
public_client = gdax.PublicClient()

buy = False
sell = False
bidding = False
asking = False

PLN = 0 
BTC = 1
PLN_min = 1e-1
BTC_min = 1e-6

cci_thr = -60
mfi_thr = 0
cmo_thr = -120

# API_p = '557c4731915037e278adcb3aa96c029a'
# API_s = 'nt/cMZQ5rvxlDrK8qhw2wtqqZD16q3VyINlTpG6+Ba8pB5SsMWpS7VcyROPqZrFjAbD/dwEi0ucNsSCWGBzOsg=='

class Strategy:
    def __init__(self, param):
        self.time = datetime.datetime.now()
        self.c1 = param['c1']
        self.c2 = param['c2'] 
        self.c2_min = 0.001
        self.n = param['n']
        self.mfi_thr = param['mfi_thr']
        self.cci_thr = param['cci_thr']
        self.cmo_thr = param['cmo_thr']
        self.buy = False
        self.sell = False
        self.bidding = False
        self.asking = False
        self.queue = 0
        self.bid = 0
        self.ask = 0
        self.name = str(self.n) + '_' + str(self.mfi_thr) + '_' + str(self.cci_thr) + '_' + str(self.cmo_thr) + '_'

    def update(self, time, trades_ohlc, orderbook_l1):
        self.time = time
        average = trades_ohlc['close'].iloc[-1]
        equilibrium = self.c1/(self.c1+self.c2*average)
        
        window = trades_ohlc.iloc[-(self.n+1):]
    
        mfi = talib.MFI(window['high'].values, window['low'].values, window['close'].values, window['size'].values, n)[-1]
        cci = talib.CCI(window['high'].values, window['low'].values, window['close'].values, n)[-1]
        cmo = talib.CMO(window['close'].values, n)[-1]
        
        if cci > self.cci_thr and mfi > self.mfi_thr and cmo > self.cmo_thr:
            sell_new = True
        else:
            sell_new = False

        if cci < -self.cci_thr and mfi < 100 - self.mfi_thr and cmo < -self.cmo_thr:
            buy_new = True
        else:
            buy_new = False

        if(equilibrium > 0.01 and equilibrium < 0.99):
            if(self.buy and not buy_new and not sell_new):
                buy_new = True
            elif(self.sell and not sell_new and not buy_new):
                sell_new = True

        self.buy = buy_new
        self.sell = sell_new

        bids = orderbook_l1['bids'][0]
        asks = orderbook_l1['asks'][0]
        best_bid = float(bids[0])
        best_ask = float(asks[0])
        difference = round(best_ask - best_bid, 2)

        if self.bidding and self.bid != best_bid:
            if self.bid > best_bid + 0.01:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#             print('remove bid')
            self.bidding = False
        elif self.bidding:
            if self.queue - float(bids[1]) > 0:
                self.queue -= float(bids[1])
            if self.queue < 0:
                self.c2 = self.c1/self.bid
                self.c1 = 0
#                 print('remove bid')
                self.bidding = False

        if(self.asking and self.ask != best_ask):
            if self.ask < best_ask - 0.01:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#             print('remove ask')
            self.asking = False
        elif self.asking:
            if self.queue - float(asks[1]) > 0:
                self.queue -= float(asks[1])
            if self.queue < 0:
                self.c1 = self.c2*self.ask
                self.c2 = 0
#                 print('remove ask')
                self.asking = False

        if(self.buy and not self.bidding and self.c1 > self.c2_min*average):
            if(difference != 0.01):
#                 print('buy', best_bid + 0.01)    
                self.bid = best_bid + 0.01
                self.queue = self.c1/average
            else:
#                 print('buy', best_bid)
                self.bid = best_bid
                self.queue = float(bids[1])
            self.bidding = True


        if(self.sell and not self.asking and self.c2 > self.c2_min):
            if(difference != 0.01):
#                 print('sell', best_ask - 0.01)
                self.ask = best_ask - 0.01
                self.queue = self.c2
                queue_my = self.queue
            else:
#                 print('sell', best_ask)       
                self.ask = best_ask
                self.queue = float(asks[1])
            self.asking = True
            
        simulation = pd.DataFrame([[self.time, self.n, self.cci_thr, self.mfi_thr, self.cmo_thr, self.c1 + self.c2*average, self.c1/average + self.c2]], \
                            columns=['time', 'n', 'cci_thr', 'mfi_thr', 'cmo_thr', 'profit_c1', 'profit_c2'])
        simulation.to_csv('Simulation/GDAX_1650_02032018_60s.csv', mode='a', header=0)
        return self.c1/average + self.c2, self.name


In [2]:
strategies = []


ns = np.linspace(10, 20, 11, dtype=np.int16)  #4
mfis = np.linspace(0, 90, 10) # 0
ccis = np.linspace(0, 90, 10) # 95
cmos = np.linspace(0, 90, 10) # 10

param= {}
c1 = 0
c2 = 1

for n in ns:
    for mfi_thr in mfis:
        for cci_thr in ccis:
            for cmo_thr in cmos:
                param['c1'] = c1
                param['c2'] = c2
                param['n'] = n
                param['mfi_thr'] = mfi_thr
                param['cci_thr'] = cci_thr
                param['cmo_thr'] = cmo_thr
                strategies.append(Strategy(param))
                
print(len(strategies))
                


# for i in range(1):
while True:    
    for attempt in range(3):
        try:
            historic_ohlc = public_client.get_product_historic_rates('BTC-EUR', granularity=60)
            orderbook_l1 = public_client.get_product_order_book('BTC-EUR', level=1)
        except:
            time.sleep(0.25)
        else:
            break
    else:
        print('Proba polaczenia nieudana')

    t = datetime.datetime.now()
        
    trades_ohlc = pd.DataFrame(data=historic_ohlc, columns=['date','low','high','open','close','size'])
    trades_ohlc.index = trades_ohlc['date'].values
    trades_ohlc = trades_ohlc.sort_index()
    
    profit_max = 0
    for strategy in strategies:
        profit, name = strategy.update(t, trades_ohlc, orderbook_l1)
        if profit > profit_max:
            profit_max = profit
            name_max = name 
    print(profit_max, name_max)
        
    print(datetime.datetime.now() - t)
    print(orderbook_l1)
    print(PLN, BTC)
    print()


11000
1.0 10_0.0_0.0_0.0_
0:01:47.760295
{'sequence': 3430813690, 'asks': [['8820.01', '3.69764336', 11]], 'bids': [['8820', '0.27309263', 3]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:15.967081
{'sequence': 3430815630, 'asks': [['8820.01', '4.00595557', 11]], 'bids': [['8820', '0.16309263', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:16.183739
{'sequence': 3430818565, 'asks': [['8818.12', '0.15502484', 2]], 'bids': [['8817.01', '0.01', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.830881
{'sequence': 3430823069, 'asks': [['8817.02', '0.21972138', 4]], 'bids': [['8817.01', '3.30986864', 9]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:20.077654
{'sequence': 3430826528, 'asks': [['8820.01', '3.28039664', 3]], 'bids': [['8820', '2.15168963', 6]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:18.975087
{'sequence': 3430830508, 'asks': [['8820.01', '0.40157561', 3]], 'bids': [['8820', '4.68299697', 7]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:17.532333
{'sequence': 3430834665, 'asks': [['8820.01', '0.81750797', 5]], 'bids': [['8820', '2.28401067', 6]]}
0 1

1.00283929585 20_60.0_0.0_40.0_
0:01:16.253400
{'sequence': 3430973063, 'asks': [['8836.83', '14.0932924', 33]], 'bids': [['8836.82', '0.01176572', 3]]}
0 1

1.00283929585 20_60.0_0.0_40.0_
0:01:16.003310
{'sequence': 3430977695, 'asks': [['8806.93', '0.03529758', 2]], 'bids': [['8800.19', '0.077841', 2]]}
0 1

1.00336576375 20_0.0_0.0_60.0_
0:01:15.839151
{'sequence': 3430981120, 'asks': [['8800.38', '0.99793961', 2]], 'bids': [['8800.37', '2.97559189', 13]]}
0 1

1.00336690389 20_0.0_0.0_60.0_
0:01:16.393683
{'sequence': 3430983617, 'asks': [['8800.38', '0.34497599', 4]], 'bids': [['8800.37', '4.03917844', 20]]}
0 1

1.00283929585 20_60.0_0.0_40.0_
0:01:16.205230
{'sequence': 3430986137, 'asks': [['8809.1', '5.08161778', 7]], 'bids': [['8809.09', '1.41092468', 6]]}
0 1

1.00283929585 20_60.0_0.0_40.0_
0:01:16.764371
{'sequence': 3430988857, 'asks': [['8809.09', '0.02783564', 3]], 'bids': [['8807.02', '0.23778799', 3]]}
0 1

1.00283929585 20_60.0_0.0_40.0_
0:01:16.417357
{'sequence': 

1.00283929585 20_60.0_40.0_0.0_
0:01:16.658286
{'sequence': 3431173127, 'asks': [['8860.66', '3.48068703', 8]], 'bids': [['8860.65', '3.68568838', 6]]}
0 1

1.00283929585 20_60.0_40.0_0.0_
0:01:16.209843
{'sequence': 3431177205, 'asks': [['8860.66', '5.63399357', 14]], 'bids': [['8860.65', '0.32644365', 6]]}
0 1

1.00283929585 20_60.0_40.0_0.0_
0:01:16.146899
{'sequence': 3431181534, 'asks': [['8855.03', '2.89615781', 9]], 'bids': [['8855.02', '0.01', 1]]}
0 1

1.00331100999 20_60.0_0.0_40.0_
0:01:17.465096
{'sequence': 3431186725, 'asks': [['8843.95', '1.58150875', 5]], 'bids': [['8843.94', '1.25659879', 5]]}
0 1

1.00331100999 20_60.0_0.0_40.0_
0:01:17.738241
{'sequence': 3431190023, 'asks': [['8843.95', '2.3682297', 5]], 'bids': [['8843.94', '2.46833369', 10]]}
0 1

1.00330987553 20_60.0_0.0_40.0_
0:01:17.675575
{'sequence': 3431193255, 'asks': [['8843.95', '0.0023', 1]], 'bids': [['8843.94', '4.77296396', 13]]}
0 1

1.0031828322 20_60.0_0.0_40.0_
0:01:17.929974
{'sequence': 3431196

1.00413993067 20_60.0_40.0_0.0_
0:01:17.901175
{'sequence': 3431387287, 'asks': [['8847.03', '0.44505945', 4]], 'bids': [['8847.02', '0.01', 1]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:17.914208
{'sequence': 3431391278, 'asks': [['8845.08', '0.97414088', 9]], 'bids': [['8845.07', '3.51248724', 14]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:17.446966
{'sequence': 3431394589, 'asks': [['8845.08', '0.80392973', 9]], 'bids': [['8845.07', '5.13218412', 23]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:18.142819
{'sequence': 3431398217, 'asks': [['8845.08', '0.42209583', 5]], 'bids': [['8845.07', '5.66778691', 24]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:17.734732
{'sequence': 3431405424, 'asks': [['8847.58', '0.52482112', 2]], 'bids': [['8847.57', '0.90022777', 5]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:17.334669
{'sequence': 3431410598, 'asks': [['8847.58', '2.60136112', 5]], 'bids': [['8847.57', '3.80873281', 14]]}
0 1

1.00413993067 20_60.0_40.0_0.0_
0:01:18.504780
{'sequence': 

1.0027893726 20_60.0_40.0_40.0_
0:01:18.824732
{'sequence': 3431640772, 'asks': [['8949.72', '0.40105444', 2]], 'bids': [['8948.59', '0.01', 1]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:19.073938
{'sequence': 3431644000, 'asks': [['8949.99', '6.17553363', 16]], 'bids': [['8949.98', '0.83161696', 7]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:18.642585
{'sequence': 3431646797, 'asks': [['8949.99', '6.11473158', 15]], 'bids': [['8949.98', '4.78189665', 12]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:17.675573
{'sequence': 3431649445, 'asks': [['8949.99', '5.14145774', 13]], 'bids': [['8949.98', '3.21272926', 14]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:18.271158
{'sequence': 3431651966, 'asks': [['8949.99', '2.30084657', 12]], 'bids': [['8949.98', '3.54354742', 17]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:18.189348
{'sequence': 3431656752, 'asks': [['8948.77', '1.21156295', 4]], 'bids': [['8948.76', '0.10781', 2]]}
0 1

1.00074437066 20_60.0_0.0_50.0_
0:01:17.558263
{'sequence': 

...

1.0 10_0.0_0.0_0.0_
0:00:56.240230
{'sequence': 3433265558, 'asks': [['9130.01', '1.27396982', 5]], 'bids': [['9130', '3.44834117', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.203708
{'sequence': 3433267212, 'asks': [['9130.01', '1.39866722', 6]], 'bids': [['9130', '3.49258137', 12]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:57.061893
{'sequence': 3433268975, 'asks': [['9130.01', '1.70275825', 8]], 'bids': [['9130', '5.4535819', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.585237
{'sequence': 3433271319, 'asks': [['9130.01', '1.45603541', 7]], 'bids': [['9130', '5.7005819', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:57.058583
{'sequence': 3433273529, 'asks': [['9130.01', '1.71048551', 9]], 'bids': [['9130', '5.29731244', 16]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.393541
{'sequence': 3433275560, 'asks': [['9130.01', '0.6073461', 7]], 'bids': [['9130', '4.94688709', 11]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.530323
{'sequence': 3433277370, 'asks': [['9130.01', '0.21409546', 4]], 'bids': [['9130', '5.41492538', 15]]}
0 1

1.

1.0 10_0.0_0.0_0.0_
0:00:56.175657
{'sequence': 3433456139, 'asks': [['9129.83', '2.23641719', 7]], 'bids': [['9129.82', '1.30638281', 4]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.794527
{'sequence': 3433458357, 'asks': [['9129.83', '5.13374939', 16]], 'bids': [['9129.82', '0.59088586', 3]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.996894
{'sequence': 3433460753, 'asks': [['9129.78', '4.51146356', 12]], 'bids': [['9129.77', '1.55841619', 2]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.210116
{'sequence': 3433463227, 'asks': [['9129.78', '3.49124718', 10]], 'bids': [['9129.77', '2.48424261', 7]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.602688
{'sequence': 3433466057, 'asks': [['9129.78', '1.2791786', 7]], 'bids': [['9129.77', '2.3936759', 9]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.669079
{'sequence': 3433469709, 'asks': [['9129.78', '0.14685678', 2]], 'bids': [['9129.77', '2.25288515', 10]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.919770
{'sequence': 3433472469, 'asks': [['9129.78', '2.23367261', 6]], 'bids': [['9129.77', '6.5876

1.0 10_0.0_0.0_0.0_
0:00:58.936506
{'sequence': 3433587602, 'asks': [['9159.48', '1.86840096', 5]], 'bids': [['9159.47', '14.18968775', 20]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:02.101114
{'sequence': 3433589102, 'asks': [['9159.48', '0.94911771', 6]], 'bids': [['9159.47', '14.38235159', 21]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.741444
{'sequence': 3433590663, 'asks': [['9159.48', '0.42762116', 6]], 'bids': [['9159.47', '14.46299903', 24]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:02.547288
{'sequence': 3433593037, 'asks': [['9160', '0.48927893', 6]], 'bids': [['9159.99', '4.21606744', 13]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:57.542932
{'sequence': 3433597251, 'asks': [['9162', '0.06296605', 2]], 'bids': [['9161.99', '1.6338288', 6]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.097748
{'sequence': 3433602836, 'asks': [['9162', '0.49024277', 4]], 'bids': [['9161.99', '4.64377885', 9]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:58.550060
{'sequence': 3433606185, 'asks': [['9166.11', '0.28581807', 2]], 'bids': [['9166.1', '2.08578872',

1.0 10_0.0_0.0_0.0_
0:00:58.669416
{'sequence': 3433694747, 'asks': [['9138.93', '0.32872576', 5]], 'bids': [['9138.92', '6.28255647', 23]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:00.500325
{'sequence': 3433697054, 'asks': [['9139', '0.75715815', 3]], 'bids': [['9138.99', '9.01991645', 20]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:00.762326
{'sequence': 3433699168, 'asks': [['9139', '0.47624333', 4]], 'bids': [['9138.99', '10.88832871', 31]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:56.254362
{'sequence': 3433701820, 'asks': [['9144', '0.01', 1]], 'bids': [['9143.99', '4.57468311', 6]]}
0 1

1.0 10_0.0_0.0_0.0_
0:01:00.822788
{'sequence': 3433705239, 'asks': [['9156.43', '0.88791809', 6]], 'bids': [['9156.42', '1.96900301', 1]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:57.403291
{'sequence': 3433709975, 'asks': [['9144.61', '0.699', 3]], 'bids': [['9144.6', '0.20214258', 3]]}
0 1

1.0 10_0.0_0.0_0.0_
0:00:59.877416
{'sequence': 3433712676, 'asks': [['9141.04', '2.91601011', 5]], 'bids': [['9141.03', '0.216363', 2]]}
0 1

1.0 

KeyboardInterrupt: 